In [ ]:
# installing dependencies
import numpy as np
import pandas as pd

from sklearn.metrics import f1_score,confusion_matrix,classification_report,mean_absolute_percentage_error as mape
from lightgbm import LGBMClassifier

In [ ]:
data=pd.read_excel('/content/WeldRight Dataset.xlsx')

In [ ]:
data.loc[0,:]

Employee Code                                   (Office id of employee)
Production                                       (Project order number)
Order Operation No    (This captures the activity to be performed by...
Date                                                 (date of activity)
Time                                       (Timestamp for the activity)
Current                                                     (In Ampere)
Humidity                                       (Relative humidity in %)
Temperature                                         (in degree celsius)
Flow                                          (in liters per min (LPM))
Job Temp                                            (in degree celsius)
Voltage                                                      (in volts)
Defect                                             (if defect occurred)
Name: 0, dtype: object

In [ ]:
data.head(5)

,Employee Code,Production,Order Operation No,Date,Time,Current,Humidity,Temperature,Flow,Job Temp,Voltage,Defect
0,(Office id of employee),(Project order number),(This captures the activity to be performed by...,(date of activity),(Timestamp for the activity),(In Ampere),(Relative humidity in %),(in degree celsius),(in liters per min (LPM)),(in degree celsius),(in volts),(if defect occurred)
1,382617,E15002965,240,2022-09-10 00:00:00,7:32:28:527,1.13,74,23,0.01,29.3,15.2,No Defect
2,382617,E15002965,240,2022-09-10 00:00:00,7:32:29:40,1.82,74,23,0,29.3,0,Tungsten Inclusion
3,382617,E15002965,240,2022-09-10 00:00:00,7:32:29:677,2.91,74,23,0,29.3,0,No Defect
4,382617,E15002965,240,2022-09-10 00:00:00,7:32:30:166,0.04,74,23,0,29.3,0,No Defect


In [ ]:
df=data.copy()

In [ ]:
df=df.drop(0,axis=0)

In [ ]:
df.head()

In [ ]:

data=data.drop(["Machine",'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
       'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24',
       'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28',
       'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32',
       'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36',
       'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40',
       'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44',
       'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48',
       'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51', 'Unnamed: 52',
       'Unnamed: 53', 'Unnamed: 54', 'Unnamed: 55', 'Unnamed: 56',
       'Unnamed: 57', 'Unnamed: 58', 'Unnamed: 59', 'Unnamed: 60',
       'Unnamed: 61', 'Unnamed: 62', 'Unnamed: 63', 'Unnamed: 64',
       'Unnamed: 65', 'Unnamed: 66', 'Unnamed: 67', 'Unnamed: 68',
       'Unnamed: 69', 'Unnamed: 70', 'Unnamed: 71', 'Unnamed: 72',
       'Unnamed: 73', 'Unnamed: 74', 'Unnamed: 75', 'Unnamed: 76',
       'Unnamed: 77', 'Unnamed: 78', 'Unnamed: 79', 'Unnamed: 80',
       'Unnamed: 81', 'Unnamed: 82', 'Unnamed: 83', 'Unnamed: 84',
       'Unnamed: 85', 'Unnamed: 86', 'Unnamed: 87'],axis=1)
data.columns

Index(['Employee Code', 'Production', 'Order Operation No', 'Date', 'Time',
       'Current', 'Humidity', 'Temperature', 'Flow', 'Job Temp', 'Voltage',
       'Defect'],
      dtype='object')

In [ ]:
data.isnull().any()  #finding null values before imputing

Employee Code         False
Production             True
Order Operation No    False
Date                  False
Time                  False
Current                True
Humidity              False
Temperature           False
Flow                  False
Job Temp              False
Voltage               False
Defect                False
dtype: bool

In [ ]:
data.Defect.unique()

array(['(if defect occurred)', 'No Defect', 'Tungsten Inclusion',
       'No Defect73', 'Tungsten Inclusion72', 'No Defect72',
       'No Defect77', 'No Defect79', 'No Defect80', 'Porosity',
       'No Defect78', 'No Defect76', 'No Defect0', 'No Defect82',
       'No Defect85', 'No Defect84', 'No Defect83', 'No Defect81',
       'No Defect75', 'No Defect86', 'Porosity86', 'No Defect87',
       'Tungsten Inclusion78', 'No Defect74', 'No Defect99', 'Porosity75',
       'Tungsten Inclusion77', 'No Defect71', 'No Defect40', 'Porosity80',
       'No Defect70', 'No Defect68', 'No Defect90',
       'Tungsten Inclusion80', 'No Defect69', 'No Defect67',
       'No Defect63', 'No Defect64', 'No Defect65', 'No Defect62',
       'No Defect61', 'No Defect60', 'No Defect59', 'Porosity76',
       'No Defect66', 'Tungsten Inclusion71', 'No Defect88',
       'Tungsten Inclusion67', 'Tungsten Inclusion75',
       'Tungsten Inclusion76', 'Porosity71', 'No Defect58', 'No Defect57'],
      dtype=object)

In [ ]:
df=data.copy()
df=df.drop(0)
df=df.drop(["Date","Time"],axis=1) #axis=1 for columns and 0 for rows

#encoding DEFECT column
df.Defect=df.Defect.replace({'No Defect':0, 'Tungsten Inclusion':1, 'No Defect73':0,
       'Tungsten Inclusion72':1, 'No Defect72':0, 'No Defect77':0,
       'No Defect79':0, 'No Defect80':0, 'Porosity':2, 'No Defect78':0,
       'No Defect76':0, 'No Defect0':0, 'No Defect82':0, 'No Defect85':0,
       'No Defect84':0, 'No Defect83':0, 'No Defect81':0, 'No Defect75':0,
       'No Defect86':0, 'Porosity86':2, 'No Defect87':0, 'Tungsten Inclusion78':1,
       'No Defect74':0, 'No Defect99':0, 'Porosity75':2, 'Tungsten Inclusion77':1,
       'No Defect71':0, 'No Defect40':0, 'Porosity80':2, 'No Defect70':0,
       'No Defect68':0, 'No Defect90':0, 'Tungsten Inclusion80':1,
       'No Defect69':0, 'No Defect67':0, 'No Defect63':0, 'No Defect64':0,
       'No Defect65':0, 'No Defect62':0, 'No Defect61':0, 'No Defect60':0,
       'No Defect59':0, 'Porosity76':2, 'No Defect66':0, 'Tungsten Inclusion71':1,
       'No Defect88':0, 'Tungsten Inclusion67':1, 'Tungsten Inclusion75':1,
       'Tungsten Inclusion76':1, 'Porosity71':2, 'No Defect58':0, 'No Defect57':0})

#encoding PRODUCTION column:
df.Production=df.Production.replace({'E15002965':1, 'E15002966':2 ,'-':4,
       15003062:5, 'E15003220':6, 'E15002881':7, 'E10002436':8, 'E50006711':9,
       'E15003200':10, 'E15003202':11, 'E15003056':12, 'E15002963':13, 'E10002437':14,
       'E15003219':15, 'E15003201':16, 'Test coopan':17, -15003217:18, -150033027:19})

#converting object type/(non numeric) to numeric type
cols=['Employee Code', 'Order Operation No','Current',
       'Humidity', 'Temperature', 'Flow', 'Job Temp', 'Voltage']
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

#imputing nuPRODUCTIONll values in ,ORDER OPERATION NUMBER,CURRENT
df.Production=df.Production.fillna(method="ffill") #ffill fills with previous values
df["Order Operation No"]=df["Order Operation No"].fillna(method="ffill")
df.Current=df.Current.fillna(method="ffill")

print(df.dtypes)
print("FINDING NULL COLUMNS:")
print(df.isnull().any())

df.head()

Employee Code           int64
Production            float64
Order Operation No    float64
Current               float64
Humidity                int64
Temperature             int64
Flow                  float64
Job Temp              float64
Voltage               float64
Defect                  int64
dtype: object
FINDING NULL COLUMNS:
Employee Code         False
Production            False
Order Operation No    False
Current               False
Humidity              False
Temperature           False
Flow                  False
Job Temp              False
Voltage               False
Defect                False
dtype: bool


,Employee Code,Production,Order Operation No,Current,Humidity,Temperature,Flow,Job Temp,Voltage,Defect
1,382617,1.0,240.0,1.13,74,23,0.01,29.3,15.2,0
2,382617,1.0,240.0,1.82,74,23,0.00,29.3,0.0,1
3,382617,1.0,240.0,2.91,74,23,0.00,29.3,0.0,0
4,382617,1.0,240.0,0.04,74,23,0.00,29.3,0.0,0
5,382617,1.0,240.0,2.51,74,23,0.00,27.8,0.0,0


Feature Engineering

In [ ]:
df1=df.copy()
df1["f2"]=np.log(df1.Voltage**0.9/df1["Job Temp"]**0.1)
df1["f1"]=np.log(df1.Voltage*0.9499-df1["Job Temp"]**0.1)+df1.f2-np.log10(df1.f2**25)
df1["f3"]=np.log(df1.Voltage*0.9499-df1["Job Temp"]**0.1) -np.log(df1.Current**0.004)
df1["f4"]=(df1["Job Temp"]<145).astype("int")
# df1["f5"]=
# corr = df1.corr()
# corr.style.background_gradient(cmap='coolwarm')
Y=df1["Defect"]
X=df1.drop(["Defect","Production","Order Operation No"],axis=1)
X=X.replace([np.inf, -np.inf], 0) #replaces infinty values with 0
X.f1=X.f1.fillna(method="ffill")
X.f2=X.f2.fillna(method="ffill")
X.f3=X.f2.fillna(method="ffill")
X.isnull().any()

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log
  result = getat

Employee Code    False
Current          False
Humidity         False
Temperature      False
Flow             False
Job Temp         False
Voltage          False
f2               False
f1               False
f3               False
f4               False
dtype: bool

In [ ]:
#correlation plot will show the dependence of variables on the target column

corr = df1.corr()
corr.style.background_gradient(cmap='coolwarm')

,Employee Code,Production,Order Operation No,Current,Humidity,Temperature,Flow,Job Temp,Voltage,Defect,f2,f1,f3,f4
Employee Code,1.000000,-0.217165,0.281897,0.031055,-0.091889,0.013019,0.020476,-0.243806,0.127009,0.011417,0.098556,-0.118850,0.011983,0.221208
Production,-0.217165,1.000000,-0.552516,0.019525,-0.166384,-0.092212,0.201547,0.220358,0.086091,-0.001044,-0.002470,0.025432,0.040770,-0.207167
Order Operation No,0.281897,-0.552516,1.000000,-0.031128,0.011687,0.062767,-0.134242,-0.285399,-0.006716,0.005776,0.088096,-0.098713,0.054192,0.263003
Current,0.031055,0.019525,-0.031128,1.000000,-0.011689,-0.009615,0.089915,0.017546,0.705117,0.059771,-0.008550,0.005085,-0.063628,-0.036545
Humidity,-0.091889,-0.166384,0.011687,-0.011689,1.000000,0.707087,-0.029554,-0.074149,-0.074754,-0.009669,-0.012912,-0.004474,-0.061425,0.070926
Temperature,0.013019,-0.092212,0.062767,-0.009615,0.707087,1.000000,-0.005525,-0.035515,-0.032865,-0.003435,-0.008345,-0.002825,-0.035948,0.035930
Flow,0.020476,0.201547,-0.134242,0.089915,-0.029554,-0.005525,1.000000,0.138386,0.137254,0.001816,-0.076593,0.070922,-0.080468,-0.127975
Job Temp,-0.243806,0.220358,-0.285399,0.017546,-0.074149,-0.035515,0.138386,1.000000,0.054911,0.013665,-0.273369,0.332609,-0.004673,-0.936062
Voltage,0.127009,0.086091,-0.006716,0.705117,-0.074754,-0.032865,0.137254,0.054911,1.000000,0.051357,0.749410,-0.562646,0.840341,-0.081066
Defect,0.011417,-0.001044,0.005776,0.059771,-0.009669,-0.003435,0.001816,0.013665,0.051357,1.000000,-0.127441,0.181768,-0.176764,-0.030629


TRAIN_TEST_SLPIT, BORDERLINE SMOTE RESAMPLING AND ML MODEL TRAINING

In [ ]:
#TRAIN_TEST SPLIT
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,random_state=1,test_size=0.1,stratify=Y)

print("y_train value counts:")
print(y_train.value_counts())
print("y_test value counts:")
print(y_test.value_counts())

#SMOTE RESAMPLING using BORDERLINE SMOTE

from imblearn.over_sampling import BorderlineSMOTE
smote=BorderlineSMOTE(sampling_strategy="not majority",n_jobs=-1,k_neighbors=1,m_neighbors=2,kind="borderline-2",random_state=42)
X_sm,Y_sm=smote.fit_resample(x_train,y_train)
print(Y_sm.value_counts())

y_train value counts:
0    739602
1      4149
2      1029
Name: Defect, dtype: int64
y_test value counts:
0    82179
1      461
2      114
Name: Defect, dtype: int64


/usr/local/lib/python3.10/dist-packages/imblearn/over_sampling/_smote/filter.py:192: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


0    739602
1    739602
2    739602
Name: Defect, dtype: int64


THE MODEL BELOW IS ABLE TO PREDICT ALL THREE CLASSES

In [ ]:
model8=LGBMClassifier(colsample_bytree=0.5177908906249942,
               learning_rate=0.21034746012122046, max_bin=1023,
               min_child_samples=19, n_estimators=968, num_leaves=331,
               reg_alpha=0.026810451163601555, reg_lambda=0.026723380640834737,
               verbose=-1,n_jobs=-1)

model8.fit(X_sm,Y_sm)
pred=model8.predict(x_test)
print("mape:",mape(y_test,pred))
print("individual f1_score:",f1_score(y_test,pred,average=None))
print("weighted f1_score:",f1_score(y_test,pred,average="weighted"))
print("confusion_matrix:")
print(confusion_matrix(pred,y_test))
print("classification_report:")
print(classification_report(pred,y_test))

mape: 7999965502857.425
individual f1_score: [0.99655685 0.37931034 0.01459854]
weighted f1_score: 0.9917656190772428
confusion_matrix:
[[82054   329   113]
 [  103   132     0]
 [   22     0     1]]
classification_report:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     82496
           1       0.29      0.56      0.38       235
           2       0.01      0.04      0.01        23

    accuracy                           0.99     82754
   macro avg       0.43      0.53      0.46     82754
weighted avg       1.00      0.99      0.99     82754



In [ ]:
# from joblib import Parallel, delayed
import joblib


# # Save the model as a pickle in a file
joblib.dump(model8, '/content/model8.pkl')

['/content/model8.pkl']

SECOND MODEL WHICH CAN PREDICT TUNGSTEN DEFECT BETTER

In [ ]:
data2=data.copy()

data2=data2.drop(0)
data2=data2.drop(["Date","Time"],axis=1)

#encoding DEFECT column
data2.Defect=data2.Defect.replace({'No Defect':0, 'Tungsten Inclusion':1, 'No Defect73':0,
       'Tungsten Inclusion72':1, 'No Defect72':0, 'No Defect77':0,
       'No Defect79':0, 'No Defect80':0, 'Porosity':2, 'No Defect78':0,
       'No Defect76':0, 'No Defect0':0, 'No Defect82':0, 'No Defect85':0,
       'No Defect84':0, 'No Defect83':0, 'No Defect81':0, 'No Defect75':0,
       'No Defect86':0, 'Porosity86':2, 'No Defect87':0, 'Tungsten Inclusion78':1,
       'No Defect74':0, 'No Defect99':0, 'Porosity75':2, 'Tungsten Inclusion77':1,
       'No Defect71':0, 'No Defect40':0, 'Porosity80':2, 'No Defect70':0,
       'No Defect68':0, 'No Defect90':0, 'Tungsten Inclusion80':1,
       'No Defect69':0, 'No Defect67':0, 'No Defect63':0, 'No Defect64':0,
       'No Defect65':0, 'No Defect62':0, 'No Defect61':0, 'No Defect60':0,
       'No Defect59':0, 'Porosity76':2, 'No Defect66':0, 'Tungsten Inclusion71':1,
       'No Defect88':0, 'Tungsten Inclusion67':1, 'Tungsten Inclusion75':1,
       'Tungsten Inclusion76':1, 'Porosity71':2, 'No Defect58':0, 'No Defect57':0})

#encoding PRODUCTION column:
data2.Production=data2.Production.replace({'E15002965':1, 'E15002966':2 ,'-':4,
       15003062:5, 'E15003220':6, 'E15002881':7, 'E10002436':8, 'E50006711':9,
       'E15003200':10, 'E15003202':11, 'E15003056':12, 'E15002963':13, 'E10002437':14,
       'E15003219':15, 'E15003201':16, 'Test coopan':17, -15003217:18, -150033027:19})

#converting object type to numeric type
cols=['Employee Code', 'Order Operation No','Current',
       'Humidity', 'Temperature', 'Flow', 'Job Temp', 'Voltage']
data2[cols] = data2[cols].apply(pd.to_numeric, errors='coerce')

#imputing null values in PRODUCTION,ORDER OPERATION NUMBER,CURRENT
# data2.Production=data2.Production.fillna(method="ffill")
# data2["Order Operation No"]=data2["Order Operation No"].fillna(method="ffill")
# data2.Current=data2.Current.fillna(method="ffill")

#dropping the rows with missing values
data2=data2.dropna()
#CREATING TARGET AND FEATURE SETS
Y=data2["Defect"]
X=data2.drop(["Defect","Production","Order Operation No","Temperature","Flow"],axis=1)
print(np.unique(Y, return_counts=True))

print(data2.dtypes)
print("FINDING NULL COLUMNS:")
print(data2.isnull().any())

data2.head()

(array([0, 1, 2]), array([491853,   2817,    704]))
Employee Code           int64
Production            float64
Order Operation No    float64
Current               float64
Humidity                int64
Temperature             int64
Flow                  float64
Job Temp              float64
Voltage               float64
Defect                  int64
dtype: object
FINDING NULL COLUMNS:
Employee Code         False
Production            False
Order Operation No    False
Current               False
Humidity              False
Temperature           False
Flow                  False
Job Temp              False
Voltage               False
Defect                False
dtype: bool


,Employee Code,Production,Order Operation No,Current,Humidity,Temperature,Flow,Job Temp,Voltage,Defect
1,382617,1.0,240.0,1.13,74,23,0.01,29.3,15.2,0
2,382617,1.0,240.0,1.82,74,23,0.00,29.3,0.0,1
3,382617,1.0,240.0,2.91,74,23,0.00,29.3,0.0,0
4,382617,1.0,240.0,0.04,74,23,0.00,29.3,0.0,0
5,382617,1.0,240.0,2.51,74,23,0.00,27.8,0.0,0


In [ ]:
df_1=data2.copy()
df_1["f2"]=np.log(df_1.Voltage**0.9/df_1["Job Temp"]**0.1)
df_1["f1"]=np.log(df_1.Voltage*0.9499-df_1["Job Temp"]**0.1)+df_1.f2-np.log10(df_1.f2**25)
df_1["f3"]=np.log(df_1.Voltage*0.9499-df_1["Job Temp"]**0.1) -np.log(df_1.Current**0.004)
df_1["f4"]=(df_1["Job Temp"]<145).astype("int")
# df1["f5"]=
# corr = df1.corr()
# corr.style.background_gradient(cmap='coolwarm')
Y1=df_1["Defect"]
X1=df_1.drop(["Defect","Production","Order Operation No"],axis=1)
X1=X1.replace([np.inf, -np.inf], 0)
X1.f1=X1.f1.fillna(method="ffill")
X1.f2=X1.f2.fillna(method="ffill")
X1.f3=X1.f2.fillna(method="ffill")
X1.isnull().any()

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log
  result = getat

Employee Code    False
Current          False
Humidity         False
Temperature      False
Flow             False
Job Temp         False
Voltage          False
f2               False
f1               False
f3               False
f4               False
dtype: bool

In [ ]:
#TRAIN_TEST SPLIT
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X1,Y1,random_state=1,test_size=0.1,stratify=Y1)

print("y_train value counts:")
print(y_train.value_counts())
print("y_test value counts:")
print(y_test.value_counts())

#SMOTE RESAMPLING

from imblearn.over_sampling import BorderlineSMOTE
smote=BorderlineSMOTE(sampling_strategy="not majority",n_jobs=-1,k_neighbors=1,m_neighbors=2,kind="borderline-2",random_state=42)
X_sm1,Y_sm1=smote.fit_resample(x_train,y_train)
print(Y_sm1.value_counts())


y_train value counts:
0    442667
1      2535
2       634
Name: Defect, dtype: int64
y_test value counts:
0    49186
1      282
2       70
Name: Defect, dtype: int64


/usr/local/lib/python3.10/dist-packages/imblearn/over_sampling/_smote/filter.py:192: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


0    442667
2    442667
1    442666
Name: Defect, dtype: int64


In [ ]:
model9=LGBMClassifier(colsample_bytree=0.5177908906249942,
               learning_rate=0.21034746012122046, max_bin=1023,
               min_child_samples=19, n_estimators=968, num_leaves=331,
               reg_alpha=0.026810451163601555, reg_lambda=0.026723380640834737,
               verbose=-1,n_jobs=-1)

model9.fit(X_sm1,Y_sm1)
pred=model9.predict(x_test)
print("mape:",mape(y_test,pred))
print("individual f1_score:",f1_score(y_test,pred,average=None))
print("weighted f1_score:",f1_score(y_test,pred,average="weighted"))
print("confusion_matrix:")
print(confusion_matrix(pred,y_test))
print("classification_report:")
print(classification_report(pred,y_test))

mape: 9273026000076.523
individual f1_score: [0.99651922 0.42666667 0.        ]
weighted f1_score: 0.9918671348520883
confusion_matrix:
[[49099   186    70]
 [   72    96     0]
 [   15     0     0]]
classification_report:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     49355
           1       0.34      0.57      0.43       168
           2       0.00      0.00      0.00        15

    accuracy                           0.99     49538
   macro avg       0.45      0.52      0.47     49538
weighted avg       1.00      0.99      0.99     49538

